KNN

princip, parameter k, metrika vzdialenosti, vyhody a nevyhody, ide o predikciu vystupu na zaklade vstupu(model pozna vstup aj vystup), ako prebieha klasifikacia,male/velke k sposobi..., citlivost na atributy, nie ucenie - len pamatanie, 

Clustering

model pozna iba vstupy a vystupy nie, model rozdeluje data na skupiny - klastre, klaster obsahuje co najpodobnejsie objekty, jednotlive klastre obsahuju co najodlisnejsie objekty, segmentacia dat a objavovanie skrytych struktur v datach, porovnanie kmeans a dbscan
TYPY:
- K-means - rozdelenie pomocou centroidov, simple, fast, pocet klastrov = k
- DBSCAN - vyuziva hustotu bodov